In [106]:
import pandas as pd
import folium
from pathlib import Path

In [141]:
# Load grid coordinates
region_name = "Napa"

grid_filenames = [
    f"{region_name}_full_safe_grid1_to_grid30_rows.xlsx",
    f"{region_name}_full_safe_grid31_to_grid60_rows.xlsx",
    f"{region_name}_full_safe_grid61_to_grid90_rows.xlsx",
    f"{region_name}_full_safe_grid91_to_grid120_rows.xlsx",
    f"{region_name}_full_safe_grid121_to_grid132_rows.xlsx"
]

grid_df = pd.DataFrame()
for grid_filename in grid_filenames:
    grid_path = Path(f"../../Grid-Coords-Files/{region_name}/{grid_filename}")
    each_grid_df = pd.read_excel(grid_path)
    grid_df = pd.concat([grid_df, each_grid_df], ignore_index=True)


print(f"Grid df shape: {grid_df.shape}")

# Load in the summary data
summary_filenames = [
    f"{region_name}_results_grid_1_to_132.csv",
]

summary_df = pd.DataFrame()
for summary_filename in summary_filenames:
    each_summary_df = pd.read_csv(f"../Output-Summary-Data/{summary_filename}")
    summary_df = pd.concat([summary_df, each_summary_df], ignore_index=True)


print(f"Summary df shape: {summary_df.shape}")

Grid df shape: (132, 5)
Summary df shape: (404, 38)


In [142]:
summary_df.columns

Index(['Room_id', 'Listing_url', 'Next_30_days_booked_days',
       'Next_30_to_60_days_booked_days', '75_rule_met', '55_rule_met',
       'Available_dates_by_year_and_month', 'Review_count_by_year_and_month',
       'Warning_level', 'Warning_type', 'Warning_message', 'Rating',
       'Accuracy_rating', 'Checking_rating', 'Cleanliness_rating',
       'Communication_rating', 'Location_rating', 'Value_rating',
       'Review_count', 'Review_months_this_year', 'Review_months_last_year',
       'Missing_review_months_this_year', 'Missing_review_months_last_year',
       'Total_missing_review_months_this_year',
       'Total_missing_review_months_last_year', 'Is_superhost', 'Guest_count',
       'Bedroom_count', 'Bed_count', 'Bath_count', 'Amenities', 'Co_hosts',
       'Highlights', 'Is_guest_favorite', 'Title', 'Latitude', 'Longitude',
       'Grid_index'],
      dtype='object')

In [143]:
summary_df = summary_df[
    (summary_df['Bedroom_count'] == 3) & 
    (summary_df['Next_30_days_booked_days'] > 15) & 
    (summary_df['Total_missing_review_months_this_year'] <= 1)
]

# Fix Summary Data (Temp)

In [144]:
def count_available_days_in_range(available_dates_dict, start_date, end_date):
    """
    Count how many days are available in a date range.
    
    Args:
        available_dates_dict: {'year': {'month': [day1, day2, ...]}}
        year: str
        month: str
        day: int
        start_date: datetime.date object
        end_date: datetime.date object (inclusive)
    
    Returns:
        int: Number of available days in the range
    """
    if not isinstance(available_dates_dict, dict) or not available_dates_dict:
        return None
    
    available_count = 0
    current_date = start_date
    
    while current_date <= end_date:
        year = str(current_date.year)   # String
        month = str(current_date.month) # String
        day = current_date.day          # Int
        
        # Check if this date is in the available dates dictionary
        if year in available_dates_dict:
            if month in available_dates_dict[year]:
                if day in available_dates_dict[year][month]:
                    available_count += 1
        current_date += timedelta(days=1)
    
    return available_count



from datetime import date, timedelta
import calendar

# Get today's date
today = date.today()

# Calculate date ranges
next_30_days_start = today
next_30_days_end = today + timedelta(days=29)  # Days 0-29 (30 days total)

next_31_to_60_start = today + timedelta(days=30)  # Day 30
next_31_to_60_end = today + timedelta(days=59)    # Day 59 (30 days total)



In [145]:
summary_df.loc[summary_df['Bath_count'] == 'Shared', 'Bath_count'] = '1'
summary_df.loc[summary_df['Bath_count'] == 'Private', 'Bath_count'] = '1'
summary_df.loc[summary_df['Bath_count'] == 'Dedicated', 'Bath_count'] = '1'
summary_df.loc[summary_df['Bath_count'] == 'No', 'Bath_count'] = '0'

summary_df['Bath_count'] = summary_df['Bath_count'].astype(float)
summary_df['Bed_count'] = summary_df['Bed_count'].astype(float)
summary_df['Bedroom_count'] = summary_df['Bedroom_count'].astype(float)

In [146]:
summary_df['Bedroom_count'].unique()

array([3.])

In [147]:
summary_df['Bath_count'].unique()

array([2.5, 3. , 2. ])

# Create Interactive Map with Color-Coded Listings

**Legend:**
- 🔴 **Red dots** = Listings that **PASS** the 75% occupancy rule  
- 🔵 **Blue dots** = Listings that **FAIL** the 75% occupancy rule  
- 🟦 **Light blue rectangles** = Search grid boundaries (with grid IDs labeled)

**Interactive Features:**
- 🔍 **Zoom**: Scroll wheel or +/- buttons
- 👆 **Pan**: Click and drag
- 💬 **Hover**: See Room ID
- 📌 **Click**: See full listing details (status badge, occupancy, reviews, ratings, location)

The popup will show a **green badge (✅)** for listings passing 75% rule or **red badge (❌)** for failing.


In [148]:
# Create interactive map
# Calculate center point of all grids
center_lat = (grid_df['ne_lat'].mean() + grid_df['sw_lat'].mean()) / 2
center_lon = (grid_df['ne_long'].mean() + grid_df['sw_long'].mean()) / 2

# Create base map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles='OpenStreetMap'
)

# Add each grid as a rectangle with detailed info
for idx, row in grid_df.iterrows():
    grid_id = row['grid_id']
    
    # Define rectangle bounds (SW corner, NE corner)
    bounds = [
        [row['sw_lat'], row['sw_long']],  # Southwest corner
        [row['ne_lat'], row['ne_long']]   # Northeast corner
    ]
    
    # Count listings within this grid
    listings_in_grid = summary_df[
        (summary_df['Latitude'] >= row['sw_lat']) & 
        (summary_df['Latitude'] <= row['ne_lat']) & 
        (summary_df['Longitude'] >= row['sw_long']) & 
        (summary_df['Longitude'] <= row['ne_long'])
    ]
    
    num_listings = len(listings_in_grid)
    num_passed_75 = listings_in_grid['75_rule_met'].sum() if num_listings > 0 else 0
    num_failed_75 = num_listings - num_passed_75
    
    # Calculate additional stats if there are listings
    if num_listings > 0:
        avg_rating = listings_in_grid['Rating'].mean()
        avg_booked_days = listings_in_grid['Next_30_days_booked_days'].mean()
    else:
        avg_rating = 0
        avg_booked_days = 0
    
    # Create detailed popup for grid
    grid_popup_html = f"""
    <div style="font-family: Arial; font-size: 12px; min-width: 250px;">
        <h3 style="margin: 5px 0; color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px;">
            📍 Grid {int(grid_id)}
        </h3>
        <table style="width: 100%; border-collapse: collapse; margin-top: 10px;">
            <tr style="background-color: #ecf0f1;">
                <td style="padding: 5px; font-weight: bold;">Total Listings:</td>
                <td style="padding: 5px; text-align: right;"><strong>{num_listings}</strong></td>
            </tr>
            <tr>
                <td style="padding: 5px; font-weight: bold;">✅ Pass 75% Rule:</td>
                <td style="padding: 5px; text-align: right; color: #27ae60;"><strong>{num_passed_75}</strong></td>
            </tr>
            <tr style="background-color: #ecf0f1;">
                <td style="padding: 5px; font-weight: bold;">❌ Fail 75% Rule:</td>
                <td style="padding: 5px; text-align: right; color: #e74c3c;"><strong>{num_failed_75}</strong></td>
            </tr>
            <tr>
                <td style="padding: 5px; font-weight: bold;">⭐ Avg Rating:</td>
                <td style="padding: 5px; text-align: right;">{avg_rating:.2f}</td>
            </tr>
            <tr style="background-color: #ecf0f1;">
                <td style="padding: 5px; font-weight: bold;">📅 Avg Days Booked:</td>
                <td style="padding: 5px; text-align: right;">{avg_booked_days:.1f} / 30</td>
            </tr>
        </table>
        <div style="margin-top: 10px; padding: 5px; background-color: #d5dbdb; border-radius: 3px; font-size: 10px;">
            <strong>Bounds:</strong><br/>
            NE: {row['ne_lat']:.5f}, {row['ne_long']:.5f}<br/>
            SW: {row['sw_lat']:.5f}, {row['sw_long']:.5f}
        </div>
    </div>
    """
    
    # Add rectangle with blue color and transparency
    folium.Rectangle(
        bounds=bounds,
        color='blue',           # Border color
        fill=True,
        fillColor='blue',       # Fill color
        fillOpacity=0.2,        # Transparency (0.2 = 20% opaque)
        weight=2,               # Border width
        popup=folium.Popup(grid_popup_html, max_width=300),
        tooltip=f"Grid {grid_id}: {num_listings} listings"
    ).add_to(m)
    
    # Add grid label at center of rectangle
    center_lat_grid = (row['ne_lat'] + row['sw_lat']) / 2
    center_lon_grid = (row['ne_long'] + row['sw_long']) / 2
    
    folium.Marker(
        location=[center_lat_grid, center_lon_grid],
        icon=folium.DivIcon(
            html=f'''
                <div style="
                    font-size: 12px; 
                    font-weight: bold; 
                    color: darkblue;
                    text-align: center;
                    text-shadow: 1px 1px 2px white, -1px -1px 2px white;
                ">
                    {int(grid_id)}
                </div>
            '''
        )
    ).add_to(m)

# Add listings as red dots
for idx, listing in summary_df.iterrows():
    # Extract listing information
    room_id = listing['Room_id']
    lat = listing['Latitude']
    lon = listing['Longitude']
    
    met_75_rule = listing['75_rule_met']
    met_55_rule = listing['55_rule_met']

    next_30_days = listing['Next_30_days_booked_days']
    next_30_to_60_days = listing.get('Next_30_to_60_days_booked_days', 'N/A')

    missing_months_this_year = listing['Total_missing_review_months_this_year']
    missing_months_last_year = listing['Total_missing_review_months_last_year']

    missing_review_months_this_year = listing['Missing_review_months_this_year']
    missing_review_months_last_year = listing['Missing_review_months_last_year']

    rating = listing['Rating']
    review_count = listing['Review_count']

    link = f"https://www.airbnb.com/rooms/{room_id}"
    
    # Determine color based on 75% rule
    if met_75_rule and (missing_months_this_year != 0):
        dot_color = 'orange'
        border_color = 'darkorange'
        status_text = '💭 Passes 75% Rule & some missing months in reviews this year'
        status_color = 'orange'  # Orange
        
    elif met_75_rule and (missing_months_this_year == 0):
        dot_color = 'red'
        border_color = 'darkred'
        status_text = '✅ Passes 75% Rule & No missing month in reviews this year'
        status_color = '#27ae60'  # Green
    else:
        dot_color = 'blue'
        border_color = 'darkblue'
        status_text = '❌ Does NOT Pass 75% Rule'
        status_color = '#e74c3c'  # Red
    
    # Create popup HTML with listing details
    popup_html = f"""
    <div style="font-family: Arial; font-size: 12px; min-width: 200px;">
        <h4 style="margin: 5px 0; color: {border_color};">Listing Details</h4>
        <div style="background-color: {status_color}; color: white; padding: 5px; margin: 5px 0; border-radius: 3px; text-align: center;">
            <strong>{status_text}</strong>
        </div>
        <table style="width: 100%; border-collapse: collapse;">
            <tr style="border-bottom: 1px solid #ddd;">
                <td style="padding: 3px; font-weight: bold;">Room ID:</td>
                <td style="padding: 3px;">{room_id}</td>
            </tr>
            <tr style="border-bottom: 1px solid #ddd;">
                <td style="padding: 3px; font-weight: bold;">Next 30 days booked:</td>
                <td style="padding: 3px;">{next_30_days} days</td>
            </tr>
            <tr style="border-bottom: 1px solid #ddd;">
                <td style="padding: 3px; font-weight: bold;">Next 30-60 days booked:</td>
                <td style="padding: 3px;">{next_30_to_60_days}</td>
            </tr>


            <tr style="border-bottom: 1px solid #ddd; background-color: #f9f9f9;">
                <td style="padding: 3px; font-weight: bold;">Months with Missing Reviews (this year):</td>
                <td style="padding: 3px;">{missing_review_months_this_year}</td>
            </tr>

            <tr style="border-bottom: 1px solid #ddd; background-color: #f9f9f9;">
                <td style="padding: 3px; font-weight: bold;">Missing months count (this year):</td>
                <td style="padding: 3px;">{missing_months_this_year}</td>
            </tr>


            <tr style="border-bottom: 1px solid #ddd; background-color: #f9f9f9;">
                <td style="padding: 3px; font-weight: bold;">Months with Missing Reviews (last year):</td>
                <td style="padding: 3px;">{missing_review_months_last_year}</td>
            </tr>
            <tr style="border-bottom: 1px solid #ddd; background-color: #f9f9f9;">
                <td style="padding: 3px; font-weight: bold;">Missing months count (last year):</td>
                <td style="padding: 3px;">{missing_months_last_year}</td>
            </tr>


            <tr style="border-bottom: 1px solid #ddd; background-color: #f9f9f9;">
                <td style="padding: 3px; font-weight: bold;">Rating:</td>
                <td style="padding: 3px;">{rating} ⭐</td>
            </tr>
            <tr style="border-bottom: 1px solid #ddd;">
                <td style="padding: 3px; font-weight: bold;">Review count:</td>
                <td style="padding: 3px;">{review_count}</td>
            </tr>
            <tr style="border-bottom: 1px solid #ddd;">
                <td style="padding: 3px; font-weight: bold;">Latitude:</td>
                <td style="padding: 3px;">{lat:.6f}</td>
            </tr>
            <tr>
                <td style="padding: 3px; font-weight: bold;">Longitude:</td>
                <td style="padding: 3px;">{lon:.6f}</td>
            </tr>
            <tr>
                <td style="padding: 3px; font-weight: bold;">Link:</td>
                <td style="padding: 3px;"><a href="{link}" target="_blank" rel="noopener noreferrer">View Listing</a></td>
            </tr>
        </table>
    </div>
    """
    
    # Add circle marker with color based on 75% rule
    folium.CircleMarker(
        location=[lat, lon],
        radius=6,                    # Size of the dot
        color=border_color,          # Border color (darkred or darkblue)
        fill=True,
        fillColor=dot_color,         # Fill color (red or blue)
        fillOpacity=0.8,             # Opacity
        weight=2,                    # Border width
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"Listing {room_id}"
    ).add_to(m)

# Count listings by 75% rule status
passed_75_rule = summary_df['75_rule_met'].sum()
failed_75_rule = len(summary_df) - passed_75_rule

print(f"✓ Created interactive map with {len(grid_df)} grids")
print(f"✓ Added {len(summary_df)} listings:")
print(f"   🔴 Red dots: {passed_75_rule} listings that PASS 75% rule")
print(f"   🔵 Blue dots: {failed_75_rule} listings that FAIL 75% rule")
print(f"📍 Map centered at: ({center_lat:.4f}, {center_lon:.4f})")
print(f"💡 Click and drag to pan, scroll to zoom in/out")
print(f"🔴 Click on dots to see listing details")

# # Display the map
# m


✓ Created interactive map with 132 grids
✓ Added 5 listings:
   🔴 Red dots: 0 listings that PASS 75% rule
   🔵 Blue dots: 5 listings that FAIL 75% rule
📍 Map centered at: (38.3143, -122.2925)
💡 Click and drag to pan, scroll to zoom in/out
🔴 Click on dots to see listing details


In [149]:
# Save map to HTML file (opens in browser - no trust issues!)
comment = "_3bedrooms"
output_file = f"./HTML/{region_name}/Filter/grid_visualization_with_listings{comment}.html"
Path(output_file).parent.mkdir(parents=True, exist_ok=True)
m.save(output_file)
print(f"✅ Map saved to: {output_file}")
print(f"📂 Open this file in your browser to view the interactive map")
print(f"\n💡 To open:")
print(f"   - Mac: Right-click → Open With → Browser")
print(f"   - Or double-click the HTML file")
print(f"\n🗺️ The HTML file has both grids and listings!")


✅ Map saved to: ./HTML/Napa/Filter/grid_visualization_with_listings_3bedrooms.html
📂 Open this file in your browser to view the interactive map

💡 To open:
   - Mac: Right-click → Open With → Browser
   - Or double-click the HTML file

🗺️ The HTML file has both grids and listings!
